In [4]:
# Extract index information with Pyserini
from pyserini.index import IndexReader
index_reader = IndexReader('indexes/lucene-index-msmarco-passage/')
index_reader.stats()

{'total_terms': 352316036,
 'documents': 8841823,
 'non_empty_documents': 8841823,
 'unique_terms': -1}

In [23]:
# Run BM25 with k1 = 0.9 and b = 0.4
!sh target/appassembler/bin/SearchCollection -hits 1000 -parallelism 4 \
 -index indexes/msmarco-passage/lucene-index-msmarco \
 -topicreader TsvInt -topics collections/msmarco-passage/queries.dev.200.tsv \
 -output runs/run.msmarco-passage.dev.200.tsv -format msmarco \
 -bm25 -bm25.k1 0.9 -bm25.b 0.4

2022-03-01 16:22:15,071 INFO  [main] search.SearchCollection (SearchCollection.java:378) - ============ Initializing Searcher ============
2022-03-01 16:22:15,073 INFO  [main] search.SearchCollection (SearchCollection.java:379) - Index: indexes/msmarco-passage/lucene-index-msmarco
2022-03-01 16:22:15,287 INFO  [main] search.SearchCollection (SearchCollection.java:459) - Language: en
2022-03-01 16:22:15,288 INFO  [main] search.SearchCollection (SearchCollection.java:460) - Stemmer: porter
2022-03-01 16:22:15,288 INFO  [main] search.SearchCollection (SearchCollection.java:461) - Keep stopwords? false
2022-03-01 16:22:15,288 INFO  [main] search.SearchCollection (SearchCollection.java:462) - Stopwords file: null
2022-03-01 16:22:15,289 INFO  [main] search.SearchCollection (SearchCollection.java:463) - Number of threads for running different parameter configurations: 1
2022-03-01 16:22:15,289 INFO  [main] search.SearchCollection (SearchCollection.java:464) - Number of threads for running ea

In [25]:
# Convert .tsv files to .trec files
!python tools/scripts/msmarco/convert_msmarco_to_trec_run.py \
 --input runs/run.msmarco-passage.dev.200.tsv \
 --output runs/run.msmarco-passage.dev.200.trec

!python tools/scripts/msmarco/convert_msmarco_to_trec_qrels.py \
 --input collections/msmarco-passage/qrels.dev.small.200.tsv \
 --output collections/msmarco-passage/qrels.dev.small.200.trec

Done!
Done!


In [29]:
# NDCG evaluation of BM25 run
!tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut \
 collections/msmarco-passage/qrels.dev.small.200.trec runs/run.msmarco-passage.dev.200.trec

ndcg_cut_5            	all	0.5278
ndcg_cut_10           	all	0.5058
ndcg_cut_15           	all	0.4980
ndcg_cut_20           	all	0.4914
ndcg_cut_30           	all	0.4884
ndcg_cut_100          	all	0.5018
ndcg_cut_200          	all	0.5332
ndcg_cut_500          	all	0.5813
ndcg_cut_1000         	all	0.6067
